In [126]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
import seaborn as sns

%matplotlib inline

In [128]:
# Load the data in
import pickle

with open("../data/processed/0202_all_data_transf.pkl", 'rb') as picklefile: 
    df = pickle.load(picklefile)

# Take a look at the datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 0 to 743
Data columns (total 21 columns):
energy_star_score                          390 non-null int64
building_floor_area                        390 non-null int64
parking_floor_area                         390 non-null int64
total_ghg_emissions                        390 non-null int64
site_eui                                   390 non-null int64
weather_normalized_site_eui                390 non-null int64
source_eui                                 390 non-null int64
weather_normalized_source_eui              390 non-null int64
water_use                                  390 non-null int64
property_id                                390 non-null int64
EMV_TOTAL                                  390 non-null float64
TAX_CAPAC                                  390 non-null float64
TOTAL_TAX                                  390 non-null float64
FIN_SQ_FT                                  390 non-null int64
YEAR_BUILT           

In [129]:
df_corr = df.corr()
df_corr

,energy_star_score,building_floor_area,parking_floor_area,total_ghg_emissions,site_eui,weather_normalized_site_eui,source_eui,weather_normalized_source_eui,water_use,property_id,...,TAX_CAPAC,TOTAL_TAX,FIN_SQ_FT,YEAR_BUILT,est_val_per_sqft,Intercept,USE1_DESC[T.Condo Garage/Miscellaneous],USE1_DESC[T.Cooperative],USE1_DESC[T.Industrial],USE1_DESC[T.Other]
energy_star_score,1.000000,0.471373,0.386254,0.259492,-0.734691,-0.727636,-0.708078,-0.709464,0.023550,0.316863,...,0.085248,0.085688,NaN,-0.094814,0.005484,NaN,-0.209816,0.207874,0.034052,-0.005250
building_floor_area,0.471373,1.000000,0.781910,0.923249,-0.414269,-0.407143,-0.281353,-0.278682,0.702187,0.085397,...,0.278893,0.279482,NaN,-0.036659,-0.053100,NaN,0.244083,-0.226747,0.039662,-0.104580
parking_floor_area,0.386254,0.781910,1.000000,0.742549,-0.407133,-0.411556,-0.228644,-0.234269,0.596335,-0.156032,...,0.136332,0.136385,NaN,-0.115163,0.018018,NaN,0.049706,-0.136802,0.021785,-0.048678
total_ghg_emissions,0.259492,0.923249,0.742549,1.000000,-0.165962,-0.164262,-0.029763,-0.028606,0.774669,0.093161,...,0.266986,0.267934,NaN,0.017354,-0.054668,NaN,0.322406,-0.281591,0.005559,-0.122838
site_eui,-0.734691,-0.414269,-0.407133,-0.165962,1.000000,0.983446,0.924804,0.921409,-0.033564,-0.005835,...,-0.104062,-0.104097,NaN,0.141789,0.071767,NaN,0.226220,-0.198696,-0.044633,-0.040686
weather_normalized_site_eui,-0.727636,-0.407143,-0.411556,-0.164262,0.983446,1.000000,0.899324,0.922025,-0.038375,-0.004245,...,-0.106717,-0.106841,NaN,0.166816,0.056315,NaN,0.246323,-0.191578,-0.045346,-0.039920
source_eui,-0.708078,-0.281353,-0.228644,-0.029763,0.924804,0.899324,1.000000,0.989293,0.024801,-0.112778,...,-0.052655,-0.052868,NaN,0.063175,0.142191,NaN,0.189217,-0.232153,-0.029699,-0.008233
weather_normalized_source_eui,-0.709464,-0.278682,-0.234269,-0.028606,0.921409,0.922025,0.989293,1.000000,0.023385,-0.114431,...,-0.052992,-0.053229,NaN,0.086790,0.132664,NaN,0.202019,-0.222819,-0.029551,-0.007261
water_use,0.023550,0.702187,0.596335,0.774669,-0.033564,-0.038375,0.024801,0.023385,1.000000,0.000145,...,0.321756,0.322588,NaN,0.016634,0.032574,NaN,0.242355,-0.281477,0.013564,-0.139197
property_id,0.316863,0.085397,-0.156032,0.093161,-0.005835,-0.004245,-0.112778,-0.114431,0.000145,1.000000,...,0.026719,0.026997,NaN,0.079875,-0.023425,NaN,0.201663,-0.036951,-0.125707,-0.079784


In [130]:
df_corr['energy_star_score'].sort_values(ascending = False)

energy_star_score                          1.000000
building_floor_area                        0.471373
parking_floor_area                         0.386254
property_id                                0.316863
total_ghg_emissions                        0.259492
USE1_DESC[T.Cooperative]                   0.207874
EMV_TOTAL                                  0.087059
TOTAL_TAX                                  0.085688
TAX_CAPAC                                  0.085248
USE1_DESC[T.Industrial]                    0.034052
water_use                                  0.023550
est_val_per_sqft                           0.005484
USE1_DESC[T.Other]                        -0.005250
YEAR_BUILT                                -0.094814
USE1_DESC[T.Condo Garage/Miscellaneous]   -0.209816
source_eui                                -0.708078
weather_normalized_source_eui             -0.709464
weather_normalized_site_eui               -0.727636
site_eui                                  -0.734691
FIN_SQ_FT   

R2: SSE/SSS is	the	portion	of variation explained by the model	

Prob (F-statisic): If p-value < 0.05, we	can	reject the null	hypothesis.

P >|t|:	if	p-value	< 0.05, we can reject the null hypothesis: This variable does contribute to this model

Prob(Omnibus): The p-value for this  test. If  p-value < 0.05, we  reject  the null  hypothesis: 
ε does  not exactly follow  the normal  distribution that  we  assumed. 

Prob(Omnibus):  The p-value for this  test.     

Jaque Beara: Normality Test

Prob(JB) Null  hypothesis:  ε is  normally  distributed. 

Skewness and Kurtosis: Idea is we  are looking for a skewness coeff.  ~ 0,  and Kurtosis  ~ 3. JB  tests if  those conditions are held  against alternatives.  

Condition Number: Note  that  is  the condi<on  number  becomes quite large,  then  this  implies that  the matrix  is  ill-posed   (does not have  a unique, well-defined  solution). This  may be  due to  multicollinear relationships  between independent variables.

In [131]:
# Fit regression model
X = df.ix[:,1:]
y = df['energy_star_score']

results = sm.OLS(y,X).fit()
# Inspect the results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      energy_star_score   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     70.04
Date:                Thu, 02 Feb 2017   Prob (F-statistic):          2.55e-107
Time:                        22:51:41   Log-Likelihood:                -1636.5
No. Observations:                 390   AIC:                             3311.
Df Residuals:                     371   BIC:                             3386.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------------------------
building_floor_area                      7.088e-05   9.48e-06      7.476      0.000      5.22e-05  8.95e-05
parking_floor_area                       2.913e-05   1.54e-05      1.890      0.060     -1.18e-06  5.94e-05
total_ghg_emissions                        -0.0014      0.001     -1.501      0.134        -0.003     0.000
site_eui                                   -0.3182      0.659     -0.483      0.629        -1.613     0.977
weather_normalized_site_eui                 0.2627      0.628      0.418      0.676        -0.973     1.499
source_eui                                  0.0800      0.336      0.238      0.812        -0.580     0.740
weather_normalized_source_eui              -0.1979      0.326     -0.608      0.544        -0.838     0.442
water_use                                  -0.0019      0.000     -6.224      0.000        -0.003    -0.001
property_id                              1.842e-11   1.78e-12     10.325      0.000      1.49e-11  2.19e-11
EMV_TOTAL                               -9.684e-07   1.09e-06     -0.888      0.375     -3.11e-06  1.18e-06
TAX_CAPAC                                   0.0002      0.000      0.734      0.464        -0.000     0.001
TOTAL_TAX                               -7.375e-05      0.000     -0.618      0.537        -0.000     0.000
FIN_SQ_FT                               -2.093e-15   4.34e-15     -0.482      0.630     -1.06e-14  6.44e-15
YEAR_BUILT                                  0.0003      0.003      0.095      0.924        -0.005     0.005
est_val_per_sqft                            0.1063      0.035      3.002      0.003         0.037     0.176
Intercept                                  43.2166      7.441      5.808      0.000        28.585    57.848
USE1_DESC[T.Condo Garage/Miscellaneous]   -13.8552      2.728     -5.079      0.000       -19.219    -8.491
USE1_DESC[T.Cooperative]                   18.2494      4.084      4.469      0.000        10.219    26.280
USE1_DESC[T.Industrial]                     5.0443      8.570      0.589      0.556       -11.808    21.897
USE1_DESC[T.Other]                          2.5188      6.201      0.406      0.685        -9.675    14.713
==============================================================================
Omnibus:                      110.056   Durbin-Watson:                   1.704
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1506.408
Skew:                           0.768   Prob(JB):                         0.00
Kurtosis:                      12.505   Cond. No.                     1.90e+28
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigen

In [132]:
# 10-fold cross-validation 
reg = LinearRegression()
scores = cross_val_score(reg, X, y, cv=10, scoring='mean_squared_error')

# scores output is negative, a sklearn quirk bc mse is used to min. optimization func.
print(-scores)

[   453.25023415   3098.24388653    210.80260051    143.64277262
    149.13488546     82.29925458   1337.20508002  63745.08618445
    630.38264083    445.87223396]


/Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and 

In [133]:
from sklearn import linear_model, preprocessing

# Standardize X
X_scaled = preprocessing.scale(X)

# Create LassoCV object; set seed for reproducibility
lasso_cv = linear_model.LassoCV(cv=5, normalize=False, verbose=True, random_state=42)

# Create model
lasso_cv.fit(X_scaled, y)

/Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................../Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might wa

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=42, selection='cyclic', tol=0.0001,
    verbose=True)

In [134]:
# Output coefficients of best model 
lasso_cv.coef_

array([ 21.0362943 ,   2.97558624,  -2.47637782,  -2.7458567 ,
        -0.        ,  -5.61422107,  -6.41242073,  -9.30110806,
         9.8558319 ,  -4.83809178,  -0.        ,  -0.        ,
         0.        ,  -0.        ,   3.98895835,   0.        ,
        -6.50371164,   4.25564834,   0.4621636 ,   0.20946661])

In [143]:
# Ridge regularization

from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
clf.fit(X_scaled, y) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [1]:
df2 = df[df['est_val_per_sqft'] > 1]
# Reg Plots 
ax = sns.regplot(x="est_val_per_sqft", y="energy_star_score", data=df2)

NameError: name 'df' is not defined